### Leitura dos dados

In [19]:
from pyspark.sql import SparkSession

In [20]:
spark = SparkSession.builder.appName('log_proj').getOrCreate()

In [21]:
data = spark.read.csv('../Data/customer_churn.csv', inferSchema=True, header=True)

In [22]:
data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [23]:
data.show()

+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|              Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|             Company|Churn|
+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|   Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|          Harvey LLC|    1|
|      Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|          Wilson PLC|    1|
|        Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...|    1|
|      Phillip White|42.0|       8010.76|              0| 6.71|     10.0|2014-04-22 12:43:12|13120 Daniel Moun...|           Smith Inc|    1|
|     

### Pré-processamento

In [24]:
data.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [25]:
selected_columns = data.select(['Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Churn'])

In [26]:
filtered_data = selected_columns.na.drop()

In [27]:
from pyspark.ml.feature import VectorAssembler, StringIndexer

In [28]:
assembler = VectorAssembler(inputCols=['Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites'], outputCol='features')

In [29]:
output = assembler.transform(filtered_data)

In [30]:
final_data = output.select('features','churn')

In [31]:
train_data, test_data = final_data.randomSplit([0.7,0.3])

### Criando o modelo

In [33]:
from pyspark.ml.classification import LogisticRegression

In [36]:
log_reg = LogisticRegression(labelCol='churn')

In [37]:
fitted_model = log_reg.fit(train_data)

24/01/03 17:52:03 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/01/03 17:52:03 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS


### Avaliando o modelo

In [38]:
summary = fitted_model.summary

In [39]:
summary.predictions.describe().show()

+-------+-------------------+-------------------+
|summary|              churn|         prediction|
+-------+-------------------+-------------------+
|  count|                628|                628|
|   mean| 0.1592356687898089|0.11942675159235669|
| stddev|0.36618737658521955|0.32454849911596606|
|    min|                0.0|                0.0|
|    max|                1.0|                1.0|
+-------+-------------------+-------------------+



In [40]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [41]:
pred_and_labels = fitted_model.evaluate(test_data)

In [42]:
pred_and_labels.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[28.0,8670.98,0.0...|    0|[8.63401916301721...|[0.99982208365591...|       0.0|
|[28.0,9090.43,1.0...|    0|[1.90091109866413...|[0.86999460927291...|       0.0|
|[30.0,7960.64,1.0...|    1|[3.44019238542473...|[0.96893730674123...|       0.0|
|[30.0,8403.78,1.0...|    0|[6.66661269526056...|[0.99872891522372...|       0.0|
|[30.0,8677.28,1.0...|    0|[4.96452625432138...|[0.99306714279348...|       0.0|
|[30.0,10744.14,1....|    1|[2.18071526308634...|[0.89850431861937...|       0.0|
|[30.0,11575.37,1....|    1|[4.46459368526382...|[0.98862158714048...|       0.0|
|[31.0,7073.61,0.0...|    0|[3.61681268721928...|[0.97383483290591...|       0.0|
|[31.0,9574.89,0.0...|    0|[3.87563914256696...|[0.97968037380310...|       0.0|
|[32.0,5756.12,0

In [43]:
evaluator = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='churn')

In [45]:
areaUnderCurve = evaluator.evaluate(pred_and_labels.predictions)

In [46]:
areaUnderCurve

0.7819819819819821

### Predição em dados não rotulados

In [47]:
new_customers = spark.read.csv('../Data/new_customers.csv', inferSchema=True, header=True)

In [48]:
new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [49]:
new_test_data = assembler.transform(new_customers)

In [50]:
new_test_data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- features: vector (nullable = true)



In [51]:
final_results = fitted_model.transform(new_test_data)

In [53]:
final_results.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [54]:
final_results.select('Company','prediction').show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+

